In [373]:
import numpy as np
import pandas as pd
import lightgbm as lgb

!pip install optuna
import optuna
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

**1. 데이터셋 생성**

In [374]:
# 데이터셋 생성

n_data = 20000

x1 = np.random.uniform(0, 1, n_data)  # 독립 변수 1
x2 = np.random.uniform(0, 2, n_data)  # 독립 변수 2
x3 = np.random.uniform(0, 3, n_data)  # 독립 변수 3

y = 3 * x1 + 2 * x2 * x3 + np.random.normal(0, 0.15, n_data)  # 종속 변수 (선형 관계 + 노이즈)

df = pd.DataFrame({'X1': x1, 'X2': x2, 'X3': x3, 'Y': y})
df['y_class'] = df['Y'].apply(lambda x: f'Class {int(x) % 4 + 1}')
df.drop(columns=['Y'], inplace=True)

In [375]:
df

,X1,X2,X3,y_class
0,0.318304,0.760519,2.594144,Class 2
1,0.530531,1.289671,1.953802,Class 3
2,0.130190,1.874153,0.835364,Class 4
3,0.433964,0.841556,1.588853,Class 4
4,0.810383,1.182842,2.423367,Class 1
...,...,...,...,...
19995,0.537864,0.084874,2.037767,Class 2
19996,0.768091,1.567483,1.897114,Class 1
19997,0.256028,0.456639,0.581464,Class 2
19998,0.008331,1.660972,1.377609,Class 1


In [376]:
# 입력 데이터를 따로 분리

X = df[['X1', 'X2', 'X3']]

In [377]:
X

,X1,X2,X3
0,0.318304,0.760519,2.594144
1,0.530531,1.289671,1.953802
2,0.130190,1.874153,0.835364
3,0.433964,0.841556,1.588853
4,0.810383,1.182842,2.423367
...,...,...,...
19995,0.537864,0.084874,2.037767
19996,0.768091,1.567483,1.897114
19997,0.256028,0.456639,0.581464
19998,0.008331,1.660972,1.377609


In [378]:
# y 를 Class 값으로 변환

y = df['y_class']

In [379]:
y

,y_class
0,Class 2
1,Class 3
2,Class 4
3,Class 4
4,Class 1
...,...
19995,Class 2
19996,Class 1
19997,Class 2
19998,Class 1


In [380]:
y.value_counts()

,count
y_class,
Class 3,5484
Class 2,5260
Class 4,4716
Class 1,4540


In [381]:
# train, valid_hpo, test 데이터로 split
# Hyperparameter Tuning 은 valid_hpo dataset 을 이용해야 함!!

from sklearn.model_selection import train_test_split

# [ Entire Dataset ] ---> [ Train + Valid_hpo ] / [ Test ]
X_train_and_valid_hpo, X_test, y_train_and_valid_hpo, y_test =\
    train_test_split(X, y,
                     test_size=0.15,
                     random_state=2025)

# [ Train + Valid_hpo ] / [ Test ] ---> [ Train ] / [ Valid_hpo ] / [ Test ]
X_train, X_valid_hpo, y_train, y_valid_hpo =\
    train_test_split(X_train_and_valid_hpo, y_train_and_valid_hpo,
                     test_size=0.15,
                     random_state=2025)

In [382]:
# 각 data의 shape 재확인

print(f'shape of X_train     : {X_train.shape}')
print(f'shape of X_valid_hpo : {X_valid_hpo.shape}')
print(f'shape of X_test      : {X_test.shape}')

shape of X_train     : (14450, 3)
shape of X_valid_hpo : (2550, 3)
shape of X_test      : (3000, 3)


In [383]:
# 향후 정확도 측정용

from sklearn.metrics import accuracy_score

**2. HyperOpt 최적화 코드**

In [384]:
# Domain Space 정의

domain_space = {
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.25)),
    'max_depth': hp.quniform('max_depth', 3, 15, q=1)
}

In [385]:
# Objective Function 정의

def objective_hyperopt(params):
    params['max_depth'] = int(params['max_depth'])

    model = lgb.LGBMClassifier(
        objective='multiclass',
        metric='multi_logloss',
        boosting_type='gbdt',
        num_iterations=100,
        verbosity=-1,
        **params
    )

    # [ Train ] ---> [ Train ] / [ Valid_lgbm ] 로 분리
    X_train_, X_valid_lgbm, y_train_, y_valid_lgbm =\
        train_test_split(X_train, y_train,
                         test_size=0.15,
                         random_state=2025)

    # LightGBM 용 eval_set 인 X_valid_lgbm, y_valid_lgbm 을 이용하여 LightGBM 모델을 valid
    model.fit(X_train_, y_train_,
              eval_set=[(X_valid_lgbm, y_valid_lgbm)])

    # HPO 용 valid set인 X_valid_hpo, y_valid_hpo 를 이용하여 하이퍼파라미터 조합 성능 테스트
    preds_hpo = model.predict(X_valid_hpo)
    accuracy_hpo = accuracy_score(preds_hpo, y_valid_hpo)

    print(f'[HPO] accuracy with {params} : {(100 * accuracy_hpo):6.2f} %')

    return {"loss": (-1) * accuracy_hpo, "status": STATUS_OK}

In [386]:
# 하이퍼파라미터 최적화 실시

trials = Trials()
best_params_hyperopt = fmin(fn=objective_hyperopt,
                            space=domain_space,
                            algo=tpe.suggest, # tpe = Tree-Structured Parzen Estimator
                            max_evals=100,
                            trials=trials)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



[HPO] accuracy with {'learning_rate': 0.021676371583853434, 'max_depth': 13} :  64.82 %
  1%|          | 1/100 [00:00<01:33,  1.06trial/s, best loss: -0.648235294117647]

/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



[HPO] accuracy with {'learning_rate': 0.10268460527195798, 'max_depth': 6} :  73.61 %
  2%|▏         | 2/100 [00:01<01:17,  1.27trial/s, best loss: -0.736078431372549]

...

[HPO] accuracy with {'learning_rate': 0.22287926820236306, 'max_depth': 3} :  55.14 %
 98%|█████████▊| 98/100 [00:53<00:00,  2.40trial/s, best loss: -0.8243137254901961]

/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



[HPO] accuracy with {'learning_rate': 0.004883156112305677, 'max_depth': 15} :  53.25 %
 99%|█████████▉| 99/100 [00:54<00:00,  2.28trial/s, best loss: -0.8243137254901961]

/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



[HPO] accuracy with {'learning_rate': 0.12025214350507642, 'max_depth': 12} :  79.33 %
100%|██████████| 100/100 [00:54<00:00,  1.82trial/s, best loss: -0.8243137254901961]


In [389]:
# 최적의 하이퍼파라미터 확인

best_params_hyperopt["max_depth"] = int(best_params_hyperopt["max_depth"])

print(f'Best Parameters : {best_params_hyperopt}')

Best Parameters : {'learning_rate': 0.24856550331704014, 'max_depth': 11}


In [390]:
# 최적의 하이퍼파라미터를 이용한 모델 학습 및 테스트

final_model = lgb.LGBMClassifier(
        objective='multiclass',
        metric='multi_logloss',
        boosting_type='gbdt',
        verbosity=-1,
        **best_params_hyperopt
)
final_model.fit(X_train, y_train)

# 평가
preds_test = final_model.predict(X_test)
accuracy_test = accuracy_score(y_test, preds_test)

print(f'[FINAL] accuracy: {(100 * accuracy_test):6.2f} %')

[FINAL] accuracy:  82.93 %


**3. Optuna 최적화 코드**

In [391]:
# Objective Function 정의

def objective_optuna(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.25),
        'max_depth': trial.suggest_int('max_depth', 3, 15)
    }

    model = lgb.LGBMClassifier(
        objective='multiclass',
        metric='multi_logloss',
        boosting_type='gbdt',
        num_iterations=100,
        verbosity=-1,
        **params
    )

    # [ Train ] ---> [ Train ] / [ Valid_lgbm ] 로 분리
    X_train_, X_valid_lgbm, y_train_, y_valid_lgbm =\
        train_test_split(X_train, y_train,
                         test_size=0.15,
                         random_state=2025)

    # LightGBM 용 eval_set 인 X_valid_lgbm, y_valid_lgbm 을 이용하여 LightGBM 모델을 valid
    model.fit(X_train_, y_train_,
              eval_set=[(X_valid_lgbm, y_valid_lgbm)])

    # HPO 용 valid set인 X_valid_hpo, y_valid_hpo 를 이용하여 하이퍼파라미터 조합 성능 테스트
    preds_hpo = model.predict(X_valid_hpo)
    accuracy_hpo = accuracy_score(preds_hpo, y_valid_hpo)

    print(f'[HPO] accuracy with {params} : {(100 * accuracy_hpo):6.2f} %')

    return accuracy_hpo

In [392]:
# 하이퍼파라미터 최적화 실시

study = optuna.create_study(direction="maximize")
study.optimize(objective_optuna, n_trials=100)

[I 2025-02-26 01:59:18,361] A new study created in memory with name: no-name-13871461-df94-4b2a-ad98-39b517b8e127
<ipython-input-391-08b556b39bea>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.25),
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[I 2025-02-26 01:59:18,791] Trial 0 finished with value: 0.8043137254901961 and parameters: {'learning_rate': 0.18921419959511526, 'max_depth': 8}. Best is trial 0 with value: 0.8043137254901961.


[HPO] accuracy with {'learning_rate': 0.18921419959511526, 'max_depth': 8} :  80.43 %


<ipython-input-391-08b556b39bea>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.25),
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[I 2025-02-26 01:59:19,106] Trial 1 finished with value: 0.41058823529411764 and parameters: {'learning_rate': 0.007531865284997157, 'max_depth': 3}. Best is trial 0 with value: 0.8043137254901961.


[HPO] accuracy with {'learning_rate': 0.007531865284997157, 'max_depth': 3} :  41.06 %


<ipython-input-391-08b556b39bea>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.25),
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[I 2025-02-26 01:59:19,563] Trial 2 finished with value: 0.7211764705882353 and parameters: {'learning_rate': 0.07003523532737758, 'max_depth': 8}. Best is trial 0 with value: 0.8043137254901961.


...

[HPO] accuracy with {'learning_rate': 0.22280868072456347, 'max_depth': 15} :  82.20 %


<ipython-input-391-08b556b39bea>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.25),
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[I 2025-02-26 02:00:05,023] Trial 98 finished with value: 0.7968627450980392 and parameters: {'learning_rate': 0.11381725655391668, 'max_depth': 15}. Best is trial 87 with value: 0.8266666666666667.


[HPO] accuracy with {'learning_rate': 0.11381725655391668, 'max_depth': 15} :  79.69 %


<ipython-input-391-08b556b39bea>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.25),
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[I 2025-02-26 02:00:05,457] Trial 99 finished with value: 0.8235294117647058 and parameters: {'learning_rate': 0.2475909040515442, 'max_depth': 15}. Best is trial 87 with value: 0.8266666666666667.


[HPO] accuracy with {'learning_rate': 0.2475909040515442, 'max_depth': 15} :  82.35 %


In [393]:
# 최적의 하이퍼파라미터 확인

best_params_optuna = study.best_params

print(f'Best Parameters : {best_params_optuna}')

Best Parameters : {'learning_rate': 0.19586203893535062, 'max_depth': 15}


In [394]:
# 최적의 하이퍼파라미터를 이용한 모델 학습 및 테스트

final_model = lgb.LGBMClassifier(
        objective='multiclass',
        metric='multi_logloss',
        boosting_type='gbdt',
        verbosity=-1,
        **best_params_optuna
)
final_model.fit(X_train, y_train)

# 평가
preds_test = final_model.predict(X_test)
accuracy_test = accuracy_score(y_test, preds_test)

print(f'[FINAL] accuracy: {(100 * accuracy_test):6.2f} %')

[FINAL] accuracy:  83.00 %
